In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as random
import scipy
from scipy import constants

In [2]:
class IsingModel:
    
    def __init__(self, grid_type='square', dim=2, pbcs=True, J=[1], Jrule='nn', starting_temp=5, field = 0):
        
        #The first step is to initialise/create various lists/value we us throughout the code
        self.grid_type=grid_type                 #we define the gridtype, dimension, boundary conditions, value of J, and initial moments
        self.dim=dim                             
        self.pbcs=pbcs                           
        self.J=J                                 
        self.num_sites = self.dim**2             
        self.Jrule=Jrule                         #the rules for which particles interact is set. In this case nn = nearest neighbours
        self.neighbours=[]                       #create a list to store the neihbours of each site
        self.init_moments = self.create_states() #create an initial array of spin states
        self.field = field
        #creating our grid
        if self.grid_type == 'square':
            self.create_square_grid()
            
        
        self.current_moments = np.copy(self.init_moments)          #here I calculate the magnetisation of the grid and add to the list
        self.magnetisation = self.current_moments.flatten().sum()  
        self.maglist = []
        self.maglist.append(self.magnetisation/self.num_sites)
        
        self.energy_list = []                                      #the same is done for energy, note(energy/magnetiation per site stored)
        self.current_energy = self.grid_energy()
        self.energy_list.append(self.current_energy/self.num_sites)
        
        
        self.temp = starting_temp
     
    def create_states(self):
        moments = np.random.choice([-1,1],(self.dim,self.dim))
        return moments
    
    def create_square_grid(self):
        
        #below a 2d grid is created, 
        x, y = np.linspace(0,self.dim-1, self.dim), np.linspace(0,self.dim-1, self.dim)
        [Y, X] = np.meshgrid(y,x)
        self.X, self.Y = X.flatten(), Y.flatten()
        alist = []
        list1 = []
        
        #here we generate the list of neighbours for each site, generated differently depending on whether periodic boundary
        #conditions are being used.
        if self.Jrule == 'nn':
            
            for i, (x1, y1) in enumerate(zip(self.X, self.Y)):

                if self.pbcs==True:
                    templist = [[ (x1-1) % self.dim, y1], [(x1+1) % self.dim, y1], [x1, (y1-1) % self.dim], [x1, (y1+1) % self.dim]]
                    reallist = [int(a[0]*self.dim+a[1]) for a in templist]
                    self.neighbours.append(reallist)

                elif self.pbcs==False:
                    templist = [[ (x1-1) , y1], [(x1+1), y1], [x1, (y1-1)], [x1, (y1+1)]]                        
                    reallist = [int(a[0]*self.dim+a[1]) for a in templist]                #this code generated neighbours
                    alist.append(reallist)
                    alist1=np.array(alist)

            if self.pbcs==False:
                for i in range(1,len(alist)):            
                    if i%self.dim == 0:
                        alist[i] = [(i+1), (i+self.dim), (i-self.dim)]               #this section removes neighbours occuring at
                                                                                     #boundary points
                    if i%self.dim == self.dim-1 and i > 1:
                        alist[i] = [(i-1), (i+self.dim), (i-self.dim)]

                for i in range(len(alist)):
                        x = np.array(alist[i]) 
                        x = x[x>=0]                                                 #grid points cannot be negative or have value
                        x = x[x<25]                                                 #greater than 25
                        list1.append(list(x))

                self.neighbours = list1
            
        elif self.Jrule == 'sq':
            for i, (x1, y1) in enumerate(zip(self.X, self.Y)):
                templist = [[ (x1-1) % self.dim, y1], [(x1+1) % self.dim, y1], [x1, (y1-1) % self.dim], [x1, (y1+1) % self.dim],[ (x1-1) % self.dim, (y1-1)%self.dim], [(x1-1) % self.dim, (y1+1)%self.dim], [(x1+1)%self.dim, (y1-1) % self.dim], [(x1+1)%self.dim, (y1+1) % self.dim]]                    
                reallist = [int(a[0]*self.dim+a[1]) for a in templist]
                self.neighbours.append(reallist)
            
    
    def single_update(self):
        moments = self.current_moments.flatten()    #I flatten the spin array so I can use a single index
        newE = self.current_energy                  #create variable to store energy during update
        if self.Jrule == 'nn' or 'sq':                     
            for i, moment in enumerate(moments):
                neighsum = np.sum(moment*moments[self.neighbours[i]])
                deltaE = 2*self.J[0] * neighsum    #change in energy after flip is twice the current energy of the site
                
                
                if (deltaE < 0):                   #if energy change is < 0 , flip the spin
                    newE += deltaE
                    moments[i] = -moments[i]
                    
                if (deltaE >= 0):                  #if not, flip if the "if" condition is met
                    if np.random.random() < np.exp((-1.0*abs(deltaE)/(self.temp))):
                        newE += deltaE
                        moments[i] = -moments[i]
                        
                        
        self.current_energy = newE                              #calulate new values of energy/mag per site and add to lists
        self.magnetisation = moments.sum()
        self.energy_list.append(newE/self.num_sites) 
        self.maglist.append(self.magnetisation/self.num_sites)
            
        
        self.current_moments =  np.reshape(moments,(self.dim,self.dim))  #reshape the array back to 2d
        return self.current_moments
        
    def grid_figure(self,moments, i):
        f = plt.figure(figsize=(5, 5), dpi=80);     
        X, Y = np.meshgrid(range(self.dim), range(self.dim))             #displaying the current grid as a meshgrid after i updates
        sp =  f.add_subplot(1,1,1)  
        plt.setp(sp.get_yticklabels(), visible=False)
        plt.setp(sp.get_xticklabels(), visible=False)      
        plt.pcolormesh(X, Y, moments, cmap="copper");
        if(i==0):
            plt.title("Initial Configuration")
        else:
            plt.title("After %d iterations" %(i));  plt.axis('tight')
            
        plt.show()
        
        
    def run(self,num_its,temp):
        self.temp = temp
        for it in range(num_its):                                         #this small function runs the single update function
            self.single_update()                                          #a given number of times
            
        
    def grid_energy(self):
        en = 0                                                            #function to calculate the total energy of the grid
        moments = self.current_moments.flatten()
        bfield = np.sum(moments)*self.field
        for i in range(self.num_sites):
            for j in range(3):
                neighsum = np.sum(moments[self.neighbours[i]])
                sightenergy = moments[i]*neighsum*(-self.J[0])
                en = en + sightenergy
                j = j+1
        if self.Jrule == 'nn':    
            en = (en) / 4 + bfield 
        elif self.Jrule == 'sq':
            en = (en) / 8 + bfield
        return en